# ベクトルであそぼう！

このワークショップでは、ベクトル化されたデータを活用して以下の実験を行います。

*   "釣った魚の名前をあてる"
*   ベクトルの可視化
*   クラスタリング
*   アノマリ検知

データをベクトル化し、機械学習のアルゴリズムを活用することで、
データ利活用の幅が広がることを実感いただけたら幸いです。

## この魚の名前は何だろう

"釣った魚の名前をあてる" ことに挑戦します

事前に、アジやサンマ、サバといった、約１００種の魚の名前を マルチモーダルAIモデル CLIPにより、埋め込みーつまりベクトル化してIRISに登録しています。(0.Prepで作業済みです)


これから行うのは、名前をあてたい魚の画像をCLIPによってベクトル化します。そして、IRISのベクトル検索により魚の画像ベクトルに最も近い魚の名前ベクトルをIRISより取得します。

それによって、魚の名前は◯◯です、とあてることができという流れとなります。

マルチモーダルモデルでベクトル化をすることにより画像とテキストの類似度を計算できること、IRISによってデータセットが大規模になっても高速にベクトル検索できることがポイントです。

![アーキテクチャ図](https://github.com/Intersystems-jp/meetup2025Workshop/blob/main/assets/2.VectorAsobo-Architecture.png?raw=true)




まず、必要なライブラリをインストールします。

In [ ]:
# 必要なライブラリをインストール
# (Transformersは、Google Colab のデフォルトでインストールされているTransformersと別のバージョンのものをインストールしています。)
!pip install -q "transformers==4.44.2"
!pip install intersystems-irispython

import iris
from transformers import AutoImageProcessor, AutoModel, AutoTokenizer
import io
import requests
from PIL import Image
import torch

IRISへの接続を定義します。

In [ ]:
# IRISへの接続情報を定義

IRIS_HOST ="20.78.1.189"
IRIS_PORT = 1972
IRIS_NAMESPACE = "USER"
IRIS_USER = "meetup2025"
IRIS_PASSWORD = "meetup2025"

IRIS_CONN_STR = f"{IRIS_HOST}:{IRIS_PORT}/{IRIS_NAMESPACE}"

# IRIS接続用関数
def get_conn():
    global _conn
    if '_conn' not in globals() or _conn is None:
        _conn = iris.connect(IRIS_CONN_STR, IRIS_USER, IRIS_PASSWORD)
    return _conn

# IRISに接続
conn = get_conn()

マルチモーダルAIモデル CLIPを準備します。(時間がかかる場合があります)

In [ ]:

# LINEヤフーさんが提供している日本語対応のCLIPモデルを利用
HF_MODEL_PATH = 'line-corporation/clip-japanese-base'
device = "cuda" if torch.cuda.is_available() else "cpu"
# トークナイザー(テキストをモデル入力用のトークンに変換する役割)の読み込み
tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_PATH, trust_remote_code=True)
# 画像前処理（リサイズ・正規化など） を行うプロセッサーを読み込み
processor = AutoImageProcessor.from_pretrained(HF_MODEL_PATH, trust_remote_code=True)
# CLIP モデル本体を読み込み、指定したデバイス（CPU/GPU）に配置
model = AutoModel.from_pretrained(HF_MODEL_PATH, trust_remote_code=True).to(device)


画像を準備し読み込みます。

In [ ]:
# 画像のURLを指定

# マサバ Image: *Scombe_japo_130302-34404_tdp.jpeg*  © Wibowo Djatmiko (Wie146), CC BY-SA 3.0
image_url = "https://upload.wikimedia.org/wikipedia/commons/d/d8/Scombe_japo_130302-34404_tdp.JPG"

#  上記のマサバ以外にも、下記の他の魚や、Web上の魚画像URLを記載して試してみましょう!

# クロマグロ - Image: *Fish4499_-_Flickr_-_NOAA_Photo_Library.jpg*  © NOAA, Public Domain
#image_url = "https://upload.wikimedia.org/wikipedia/commons/9/99/Fish4499_-_Flickr_-_NOAA_Photo_Library.jpg"

# ギンブナ - Image: *Gin–buna.jpg*  CC BY-SA 3.0
#image_url = "https://upload.wikimedia.org/wikipedia/commons/d/d9/Gin%E2%80%93buna.jpg"

# キンメダイ - Image: *Splendid_alfonsino_(_Beryx_splendens_).jpg*  © NOAA's Fisheries Collection , SEFSC Pascagoula Laboratory; Collection of Brandi Noble, NOAA/NMFS/SEFSC - http://www.photolib.noaa.gov/htmls/fish4120.htm, Public Domain
#image_url = "https://upload.wikimedia.org/wikipedia/commons/7/7c/Splendid_alfonsino_%28_Beryx_splendens_%29.jpg"

# マハタ - Image: *Epinephelus_septemfasciatus_Enosui1.jpg*  © Totti, CC BY-SA 4.0
#image_url = "https://upload.wikimedia.org/wikipedia/commons/d/da/Epinephelus_septemfasciatus_Enosui1.jpg"

# マアジ - Image: *Trachurus_japonicus_Umi-Tamago.jpg*  © Totti, CC BY-SA 4.0
#image_url = "https://upload.wikimedia.org/wikipedia/commons/4/4c/Trachurus_japonicus_Umi-Tamago.jpg"

# 画像を読み込む
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(image_url, headers=headers)
response.raise_for_status()

# 画像データとして開く
image = Image.open(io.BytesIO(response.content))

# 画像サイズを指定
image.thumbnail((500, 300))
# 画像を表示
image.show()
image

画像をベクトル化します。

In [ ]:
# 画像をモデルの入力用テンソル形式に変換
image = processor(image, return_tensors="pt").to(device)

# 画像特徴量（ベクトル）を抽出
with torch.no_grad():
    image_features = model.get_image_features(**image)


魚の名前をあてる - ベクトル検索で、写真のベクトルに類似するテキストベクトルを取得します。

In [ ]:
# ベクトルのフォーマット変換 - PytorchテンソルをFloatのリストにする
float_list = image_features[0].cpu().tolist()

# ベクトルのフォーマット変換 - カンマ区切りの文字列にする
emb_str = ",".join(map(str, float_list))

### 特徴ベクトルを引数として、類似するものを取得する関数
def get_similar_desc(conn, emb_str):
    try:
        # 引数のベクトルに類似するベクトルを３件取得する。コサイン類似度(VECTOR_COSINE)で類似度を計算している。
        sql = ("SELECT TOP 3 Name,VECTOR_COSINE(Features,TO_VECTOR(?,double)) FROM MeetUp2025.Fish fish "
        "ORDER BY VECTOR_COSINE(Features,TO_VECTOR(?,double)) DESC"
              )

        with conn.cursor() as cursor:
            cursor.execute(sql, [emb_str, emb_str])
            results = cursor.fetchall()
            retrieved_info = "\n".join(f"【魚の名前候補】 {row[0]} 【類似度スコア】 {row[1]}" for row in results)

        return retrieved_info

    except Exception as e:
        return f"Error: {str(e)}"

# IRISのベクトル検索により、指定画像に類似する魚名を取得
info = get_similar_desc(conn, emb_str)
print(info)


いかがでしたでしょうか？

100% の精度ではありませんが、
画像から魚名をあてることができているのはないでしょうか。

ぜひいろいろな魚画像で試してみてください！

# ベクトルの可視化

ベクトルを可視化する利点はいくつかあります。

たとえば、ベクトル検索をしたが意図した通りに検索できない際に、モデルがどのようにデータ同士を捉えているかを把握することができます。
また、データの傾向を人に伝える・・レポートする際にも有用となります。

ベクトルの可視化を行う前に、まずはIRISにすでに登録されている魚の名前ベクトルをそのまま表示してみましょう。

In [ ]:
try:
  sql = ("SELECT TOP 1 Name, Features FROM MeetUp2025.Fish fish")

  with conn.cursor() as cursor:
      cursor.execute(sql)
      row = cursor.fetchone()
      retrieved_info = f"{row[0]} のベクトルをそのまま表示します： {row[1]}"

  print(retrieved_info)

except Exception as e:
  print(f"Error: {str(e)}")

ベクトルは、数字の配列となっていますね。AIモデルが出力するベクトルは数百から数万の次元からなる数値であり、人間がみても理解できるものではありません。

これらを可視化するためには、ベクトルの次元数を人間に親しみやすい次元数・・2次元や3次元に削減する必要があります。

今回はベクトルの次元を削減して3次元にし、3次元空間にプロットしてみましょう。

ここでは、魚の名前をあてる実験で利用した魚名ベクトルの可視化を行います。

In [ ]:
# 必要なモジュールをインポート
import umap.umap_ as umap
import numpy as np
import plotly.express as px
import pandas as pd
import plotly.io as pio

In [ ]:

# 文字列からfloatリストへ変換する関数
def parse_vector_string(s):
    return [float(x) for x in s.split(',')]

# IRISに格納された魚の名前 (もしくは画像)のベクトルを取得する関数
def get_vectors(conn, modality):
    try:
        table = 'MeetUp2025.Fish' if modality == 1 else 'MeetUp2025.FishImage'

        sql = f"SELECT Name, Features FROM {table} fish"

        with conn.cursor() as cursor:
            cursor.execute(sql)
            results = cursor.fetchall()
        return results

    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
modality = 1 # text

# IRIS から魚の名前ベクトルを取得
info = get_vectors(conn, modality)

# ベクトルとラベルを分離
labels = [row[0] for row in info]

# ベクトルの変換 (文字列からfloatに変換)
vectors = [parse_vector_string(row[1]) for row in info]

In [ ]:
# ３次元のベクトルを可視化する関数  (Plotlyを利用してインタラクティブな表示が可能)
def visualizeVectors3dInteractive (labels, vectors_3d):
    pio.renderers.default = 'colab' # Google Colabで開くことを指定

    # 表示のしかたを指定
    # pandas の DataFrame に変換し、列名 "x", "y", "z" を設定
    df_3d = pd.DataFrame(vectors_3d, columns=["x", "y", "z"])

    # 各ベクトルに対応するラベルを "label" 列として追加
    df_3d["label"] = labels

    # 3D散布図を定義
    fig = px.scatter_3d(df_3d, x="x", y="y", z="z", text="label")

    # 各点（マーカー）のサイズを変更 - デフォルトより小さめの「4」に設定
    fig.update_traces(marker=dict(size=4))

    # 3Dグラフをインタラクティブに表示
    fig.show()

ベクトルの次元削減にはUMAPというアルゴリズムを利用します。
UMAPは、データ同士の意味的な距離をある程度保持しながら、少ない次元に変換してくれるアルゴリズムです。
次元削減アルゴリズムは有名なものがいくつかありますが、UMAPの特徴としては、比較的少ない計算量で、大規模なデータセットにも対応できるといわれている人気のあるアルゴリズムです。

UMAPのパラメーターには、下記のようなものがあります。

*   n_components: 次元数を指定します。今回は3次元にしたいので、3を指定します。
*   n_neighbors: 各点の近傍のサイズを指定します。小さくすると、局所構造が強調され、クラスタが分離されやすくなります。 大きくすると、全体の配置が保存されやすくなります。
*   min_dist: 埋め込み空間での最小距離です。 小さくすると密集し、大きくすると散らばる配置となります。
*   metric: 距離の測定方法。ここでは「コサイン類似度（cosine similarity）」を使います。ベクトル方向の違いを基準に距離を測ります。







In [ ]:
# UMAPの設定を行う - 次元数などを指定
umap_model_3d =  umap.UMAP(n_components=3, n_neighbors=15, min_dist=0.2, metric='cosine')

In [ ]:
# テキストベクトルの次元を削減して 3次元に変換する
reduced_3d = umap_model_3d.fit_transform(vectors)

これで、ベクトルの次元が削減されました。ベクトルを表示してみましょう。

In [ ]:
# 次元削減後のベクトルを5件表示してみます
print(reduced_3d[0:5])

ベクトルは、3つの数値であらわされるようになりました。つまり3次元に削減されました。

では3D空間にベクトルを描画します。

グラフの描画には、Pythonのライブラリ・Plotlyを利用します。
グラフが描画されると、マウスホイール(もしくはトラックパッド)の操作で、Zoom In/Outの操作が可能です。また表示枠の右上にマウスを近づけると操作用のコントローラが表示され、Zoomや画像のダウンロード等が可能です。

In [ ]:
# テキストベクトルの 3D 可視化
#  Plotly により3次元空間にデータが描画されます。
# マウスホイール(もしくはトラックパッド)の操作でZoom In/Outも可能です。
visualizeVectors3dInteractive (labels, reduced_3d)

似た魚は近い場所にプロットされていますでしょうか？

他にどのような気づきを得られましたでしょうか？

このように、可視化をすることでモデルがデータをどう解釈しているかをより直感的に把握することができます。


次元数の削減にはUMAPを利用しましたが、n_neighborsやmin_distのパラメータを変化させると、可視化した際に各点の表示のされかたも変化します。ぜひ試してみてください。


# イメージ ベクトルの可視化

魚名ベクトルの可視化につづき、次は画像から取得したベクトルの可視化を行います。

In [ ]:
modality = 2 # image

# IRIS に格納された画像ベクトルを取得
image_info = get_vectors(conn, modality)

# ベクトルとラベルを分離
image_labels = [row[0] for row in image_info]

# ベクトルの変換 (文字列からfloatに変換)
image_vectors = [parse_vector_string(row[1]) for row in image_info]

# 次元削減して3次元に変換する
image_reduced_3d = umap_model_3d.fit_transform(image_vectors)

# 画像ベクトルの 3D 可視化
visualizeVectors3dInteractive (image_labels, image_reduced_3d)

画像もテキストと同様に、似たものは近く、異なっているものは遠くに配置されることを確認できたかと思います。

(画像の背景等の影響を受けるため、直感的な想定とは若干異なった配置となる可能性はあります）

テキスト、画像ともに同じモデルを利用しています。同じモデルで異なる形式を扱えるのが、マルチモーダルモデルの特徴となります。

# イメージとテキストのベクトルを同じ空間で可視化

ここまでで、マルチモーダルAIモデル・CLIPによってベクトル化した魚名ベクトル(テキスト)、魚の画像ベクトルを可視化してきました。

テキストと画像の類似度を計算できることがCLIPの特徴ですが、
これら２つを同時に可視化するとどうなるでしょうか？

In [ ]:
# ベクトルの統合(画像+テキスト)
multi_modal_vectors = vectors + image_vectors
multi_modal_labels = labels + image_labels

In [ ]:
# UMAPの設定を行う - 次元数の指定や密集度の調整
umap_model_3d =  umap.UMAP(n_components=3, n_neighbors=30, min_dist=0.3, metric='cosine')

# 画像+テキスト ベクトルを次元削減して3次元に
multi_modal_reduced_3d = umap_model_3d.fit_transform(multi_modal_vectors)

In [ ]:
# 画像+テキスト ベクトルの 3D 可視化
visualizeVectors3dInteractive (multi_modal_labels, multi_modal_reduced_3d)

テキストと画像のベクトルは離れた場所にプロットされました。

これはモダリティギャップといわれ、マルチモーダルを扱う際に汎用的にみられる現象です。

同じモーダル同士 - 画像と画像、テキストとテキストの類似度は高く出やすく、異なるモーダル間 - 画像とテキストでは、同じ意味を持っていても数値的な類似度が低めに出ます。

これは、画像とテキストの特徴表現がもともと性質の異なる情報であることに起因します。
画像はピクセルパターンから抽出された視覚的特徴であり、テキストは単語や文脈から抽出された意味的特徴となります。


両者を同じ空間に揃えるよう学習しても、完全には一致せず「ズレ」が残ってしまいます。これがモダリティギャップです。


このギャップを補正するような研究がされていて、今後さらに高精度なマルチモーダルモデルが期待されます。

# クラスタリング

クラスタリングは似たもの同士をグループ分けするアルゴリズムです。

顧客セグメンテーションや製品の自動仕分けなど、様々なユースケースがあります。

ベクトルを可視化した際に、似たデータ同士が近くに集まっているのを実感できたと思いますが、クラスタリングによりグループ分けをしてみましょう。


ここでは、クラスタリングの代表的なアルゴリズム K-Meansを利用して、魚名ベクトルのクラスタリングをします。
K-Meansは、データをK個にグループ分けするアルゴリズムです。

In [ ]:
# 必要なライブラリをロード
from sklearn.cluster import KMeans

# クラスタ数(K)を設定
n_clusters = 5

# K-Menasを設定
kmeans = KMeans(n_clusters=n_clusters, random_state=5, n_init=10)

# K-Meansによりクラスタリングを実行
cluster_labels = kmeans.fit_predict(vectors)

In [ ]:
### クラスタ可視化用の関数
def visualizeVectors3dInteractive_with_clusters(labels, vectors_3d, cluster_labels):
    pio.renderers.default = 'colab' # Google Colabで実行する場合

    # 表示の仕方を指定
    df_3d = pd.DataFrame(vectors_3d, columns=["x", "y", "z"])
    df_3d["label"] = labels

    #クラスタ番号の指定
    df_3d["cluster"] = cluster_labels #クラスタ番号の指定

    # 3D散布図（クラスタごとに色分け）
    fig = px.scatter_3d(
        df_3d,
        x="x", y="y", z="z",
        color="cluster",              # クラスタごとに色分け
        text="label",                 # ラベル表示
        symbol="cluster",             # クラスタごとにマーカーも変える
    )

    # 点のサイズを調整
    fig.update_traces(marker=dict(size=5))

    # グラフタイトルと凡例（クラスタの説明）のタイトルを設定
    fig.update_layout(title="3D UMAP (with Clusters)", legend_title="Cluster")

    # Plotlyにより3Dグラフを描画して表示
    fig.show()

In [ ]:
# クラスタの可視化を実行
visualizeVectors3dInteractive_with_clusters(labels, reduced_3d, cluster_labels)

データがK個に分けられ、クラスタリングされていますでしょうか？

クラスタをもう少し細かく分けたいと感じられた場合は、ぜひ、Kの数にもう少し大きいものを指定して再度実行してみてください。


K・・クラスタの数は、データをどのくらいの粒度でグループ分けしたいかによって決まります。

Kが小さすぎると、各クラスタが広すぎて「違うもの」が混ざります。
Kが大きすぎると、細かすぎて「似たもの」が別クラスタに分かれます。

Kの数をいくつにするかはいろいろな議論があり絶対的な正解があるものではなく、データの性質やクラスタリングの目的によって異なります。

一般的には、Kの数を自動で計算する、エルボー法やシルエット法がよく利用されます。


今回は、エルボー法を利用してKの数を計算してみましょう。

### エルボー法によるKの数の算出

エルボー法（Elbow Method）は、「クラスタ数 K を増やすとどこまでクラスタリングの精度が上がるか」を見て、
「これ以上 K を増やしても効果が薄い」地点を探す方法です。
その形が“ひじ（elbow）”に似ていることからこの名前がついています。

#### 直感的なイメージ

K-Meansでは、クラスタ数を増やすほど
各点がより近いクラスタに割り当てられるので、
クラスタ内の誤差（SSE: Sum of Squared Errors, 残差平方和）は小さくなります。

ところが、Kを増やし続けると改善の度合いがだんだん小さくなります。
（＝「Kを増やしてももう大して良くならない」状態）

その「改善が鈍化する曲がり角」＝ 肘（Elbow）の位置が、
最適なKの目安になります。

In [ ]:
# エルボーの“曲がり点”を自動検出するライブラリ kneed をインストール
!pip -q install kneed
from kneed import KneeLocator

# K を 2〜29 で試す（範囲はタスクに応じて調整）
Ks = list(range(2, 30))
inertias = []


for k in Ks:
    # K-Meansを実施
    km = KMeans(n_clusters=k, random_state=5, n_init=10)
    km.fit(vectors)

    # 各Kでの SSE（クラスタ内誤差平方和）を格納
    # K-MeansでのSSE＝各点が自分のクラスタ中心から離れている“合計の二乗距離”。小さいほどクラスタ内がまとまっていることを示す
    inertias.append(km.inertia_)  # SSE

# K–SSE の折れ線を描画
fig = px.line(
    x=Ks, y=inertias, markers=True,
    labels={"x": "K (クラスタ数)", "y": "SSE（Within-Cluster Sum of Squares）"},
    title="エルボー法：KとSSEの関係"
)
fig.update_layout(xaxis=dict(dtick=1)) # dtick=1: x 目盛りを整数刻みに

# 曲がり点の自動検出
#  curve="convex"：K–SSE の関係は一般に凸（convex）形状（最初は急降下→徐々に緩やか）
#  irection="decreasing"：SSE は 減少 していく指標
# knee.knee には エルボーとなるKの値が入る

knee = KneeLocator(
    Ks, inertias, curve="convex", direction="decreasing"
)
print("推定されたエルボー(K):", knee.knee)

# 図にエルボーを追加
if knee.knee is not None:
    import plotly.graph_objects as go
    fig.add_trace(go.Scatter(
        x=[knee.knee], y=[inertias[Ks.index(knee.knee)]],
        mode="markers+text", text=[f"Elbow = {knee.knee}"],
        textposition="top center"
    ))
    fig.show()

曲がり点、エルボーとなるKの数を得られましたでしょうか。

エルボーとなるKの数で、再度クラスタリングを行い、可視化してみましょう。

In [ ]:
# クラスタ数(K)を再設定

# クラスタの数を記載してください
n_clusters = ?    # <== ? を数値に書き換えてください
kmeans = KMeans(n_clusters=n_clusters, random_state=5, n_init=10)
cluster_labels = kmeans.fit_predict(vectors)

# クラスタの可視化を実行
visualizeVectors3dInteractive_with_clusters(labels, reduced_3d, cluster_labels)

エルボー法によって得られたKの数で、データはどのようにクラスタリングされましたでしょうか？



# アノマリ検知

魚の名前をIRISにあらかじめ登録していますが、魚(や海の生き物)ではないものも紛れ込んでしまっています。
それらを検知してみましょう。

### アノマリ検知SQL

アノマリ検知には様々な方法がありますが、ここでは、お手本となるデータをいくつか指定し、それらのお手本のいずれにも類似していないものをアノマリとして検知する、というお手軽な(?)アノマリ検知となります。

ここでは、マアジ、ボタンエビ、マダコをお手本としてこれらとの類似度のうち最大値 をMOST_SIMALARとし、MOST_SIMALARが低いものを抽出します。
これにより、お手本データのいずれにも類似していないデータを抽出します。


![アノマリ検知SQLのイメージ図](https://github.com/Intersystems-jp/meetup2025Workshop/blob/main/assets/2.VectorAsobo-AnomalyDetection.png?raw=true)


In [ ]:
sql = f"""
SELECT
  TOP 5 id, name,
  GREATEST( VECTOR_COSINE(features, (SELECT features from MeetUp2025.Fish where name='マアジ')),
  VECTOR_COSINE(features, (SELECT features from MeetUp2025.Fish where name='ボタンエビ')),
  VECTOR_COSINE(features, (SELECT features from MeetUp2025.Fish where name='マダコ'))) AS MOST_SIMILAR -- マアジ、ボタンエビ、マダコとの類似度のうち最も高い類似度(MOST_SIMALAR)を算出
FROM
  MeetUp2025.Fish fish
HAVING
  MOST_SIMILAR < 0.6
ORDER BY
  MOST_SIMILAR -- MOST_SIMALARが低い順に並び替え
ASC
"""
try:
  with conn.cursor() as cursor:
      cursor.execute(sql)
      results = cursor.fetchall()
      retrieved_info = "\n".join(f"【アノマリ候補】 {row[1]} 【MOST_SIMILARスコア】 {row[2]}" for row in results)

  print(retrieved_info)
except Exception as e:
  print(f"Error: {str(e)}")

いかがでしたでしょうか？ 魚や海の生き物以外のデータを検出できましたでしょうか。

ベクトル検索で"類似しているもの"を検索するのではなく、
"類似していないもの"を検索することで、アノマリ検知に応用する例でした。

## Isoration Forestによる異常検知

もう一つ、アルゴリズムによるアノマリ検知を試したいと思います。

Isolation Forest（アイソレーションフォレスト）は、ランダムな分割を繰り返すことで「孤立しやすい点」を異常として検出する手法です。

● 仕組みのイメージ

* ランダムに特徴を選び、ランダムな閾値でデータを分割する木（Isolation Tree）を作る。

* 通常の点（密集している点）は、たくさんの分割をしないと孤立しない。

* 一方、異常点（他から離れた点）は、少ない分割回数で孤立する。

* これを多数の木（フォレスト）で繰り返し、平均的な分割回数が短い点ほど「アノマリ」と判断する。

つまり、「他の点から孤立しやすい点」を異常とみなすアルゴリズム
です。

![アイソレーションフォレストのイメージ図](https://github.com/Intersystems-jp/meetup2025Workshop/blob/main/assets/2.VectorAsobo-IsorationForest.png?raw=true
)

In [ ]:
# ライブラリをロード
from sklearn.ensemble import IsolationForest

# 全データのうち、約3%を「異常」とみなす想定
iso = IsolationForest(contamination=0.03, random_state=5)

# fit_predict は fit（学習）と predict（予測）を同時に行うメソッド
# anomaly_scoresには -1: アノマリ, 1: 通常が入る
anomaly_scores = iso.fit_predict(vectors)

# 結果を可視化する
visualizeVectors3dInteractive_with_clusters(labels, reduced_3d, anomaly_scores)

Isolation Forestによるアノマリ検知の結果はいかがでしたでしょうか？



*※ なお、ブラックタイガーがアノマリと判定される場合がありますが、その名のとおり、黒い虎とも捉えられるから、というのは私の推測です。どう思われますか？*


### 後処理

In [ ]:
# IRIS との接続を閉じる
_conn.close()
_conn = None

## ベクトルであそぼう！まとめ

ベクトル検索・可視化・クラスタリング・異常検知を通して、AIモデルがデータをどのように捉えているかをみてきました。

実際に機械学習アルゴリズムを扱う際には、より大規模で複雑なデータを扱い、前処理や精度評価、パラメータ最適化、モデル再学習などの工程を通して性能と信頼性を高める必要がありますが、今回はカジュアルにイメージをつかんでいただくようなコードとなっております。

開発者の皆様のデータ利活用のアイディアにほんの少しでも寄与できましたら幸いです。
今後、皆様とデータ利活用について語れることを楽しみにしております！

それでは、引き続きワークショップをお楽しみください！！
